In [1]:
import numpy as np
import pandas as pd

In [2]:
def load_original_data():
    file1 = pd.read_csv('Data/file1.csv')
    file2 = pd.read_csv('Data/file2.csv')
    file3 = pd.read_csv('Data/file3.csv')
    file3.rename(columns={'State': 'ST', 'Gender': 'GENDER'}, inplace=True)
    return pd.concat([file1,file2,file3],ignore_index=True)

In [3]:
df = load_original_data()

In [4]:
df.shape

(12074, 11)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12074 entries, 0 to 12073
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Customer                   9137 non-null   object 
 1   ST                         9137 non-null   object 
 2   GENDER                     9015 non-null   object 
 3   Education                  9137 non-null   object 
 4   Customer Lifetime Value    9130 non-null   object 
 5   Income                     9137 non-null   float64
 6   Monthly Premium Auto       9137 non-null   float64
 7   Number of Open Complaints  9137 non-null   object 
 8   Policy Type                9137 non-null   object 
 9   Vehicle Class              9137 non-null   object 
 10  Total Claim Amount         9137 non-null   float64
dtypes: float64(3), object(8)
memory usage: 1.0+ MB


#### Duplicate row cleaning

In [6]:
show_duplicates = df[df.duplicated()]
show_duplicates

,Customer,ST,GENDER,Education,Customer Lifetime Value,Income,Monthly Premium Auto,Number of Open Complaints,Policy Type,Vehicle Class,Total Claim Amount
1072,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1073,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1074,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1075,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1076,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
4006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4008,GS98873,Arizona,F,Bachelor,323912.47%,16061.0,88.0,1/0/00,Personal Auto,Four-Door Car,633.6
4009,CW49887,California,F,Master,462680.11%,79487.0,114.0,1/0/00,Special Auto,SUV,547.2


In [7]:
df = df.drop_duplicates()

In [8]:
df.shape

(9135, 11)

In [9]:
# underscores

df.columns = df.columns.str.replace(' ', '_')

In [10]:
# lower case

df= df.rename(columns=str.lower)

In [11]:
# Check the datatype of resultant df

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9135 entries, 0 to 12073
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   customer                   9134 non-null   object 
 1   st                         9134 non-null   object 
 2   gender                     9012 non-null   object 
 3   education                  9134 non-null   object 
 4   customer_lifetime_value    9127 non-null   object 
 5   income                     9134 non-null   float64
 6   monthly_premium_auto       9134 non-null   float64
 7   number_of_open_complaints  9134 non-null   object 
 8   policy_type                9134 non-null   object 
 9   vehicle_class              9134 non-null   object 
 10  total_claim_amount         9134 non-null   float64
dtypes: float64(3), object(8)
memory usage: 856.4+ KB


#### Customer Lifetime Value [Cleaning]

In [12]:
df = df.drop(columns = 'customer')

In [13]:
df['customer_lifetime_value'].isna().value_counts()

False    9127
True        8
Name: customer_lifetime_value, dtype: int64

for idx, row in df.iterrows():
    if type(df['customer_lifetime_value']) == str:
        df.at[idx, 'customer_lifetime_value'] = int(float(df['customer_lifetime_value'].rstrip('%')))
    


In [14]:
df['customer_lifetime_value'] = df['customer_lifetime_value'].fillna(0)

In [15]:
df['customer_lifetime_value'].isna().value_counts()

False    9135
Name: customer_lifetime_value, dtype: int64

In [16]:
df['customer_lifetime_value'] = pd.to_numeric(df['customer_lifetime_value'], errors='coerce')


In [17]:
df['customer_lifetime_value'].isna().value_counts()

False    7078
True     2057
Name: customer_lifetime_value, dtype: int64

In [18]:
def remove_charac():
    
    """
    returns a list removing % character from the subset of column containing string values
    """
    lst = []
    for cell in df['customer_lifetime_value']:
        if type(cell) == str:
            cell = int(cell.rstrip("%"))
            lst.append(cell)
    return lst

    
    
    

#### Number of Open Complaints [Cleaning]

In [19]:
df['number_of_open_complaints'] = df['number_of_open_complaints'].fillna(df['number_of_open_complaints'].mode())


In [20]:
df = df[df['number_of_open_complaints'].notnull()]
print(df['number_of_open_complaints'].isna().value_counts())

False    9134
Name: number_of_open_complaints, dtype: int64


In [21]:
#This splits the Dataframe (all rows) for every occurrence of '/' 
# and then takes the value between 1st and 2nd '/'.

df['number_of_open_complaints_x'] = df['number_of_open_complaints'].str.rsplit(pat = '/')


In [22]:
df['number_of_open_complaints_x'] = df['number_of_open_complaints_x'].str[1]


#### Filtering data and Correcting typos

In [26]:
df['st']=np.where([i.startswith("Cali") for i in df['st']],'California',df['st'])
df['st']=np.where([i.startswith("AZ") for i in df['st']],'Arizona',df['st'])
df['st']=np.where([i.startswith("WA") for i in df['st']],'Washington',df['st'])


In [39]:
df['gender']=np.where([i.startswith("Male") for i in df['gender']],'M',df['gender'])
df['gender']=np.where([i.startswith("female") for i in df['gender']],'F',df['gender'])
df['gender']=np.where([i.startswith("Femal") for i in df['gender']],'F',df['gender'])